In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=7f973d25659923d561694e03013cbd0e1916fc57fe961fc8e759bbd217f99c16
  Stored in directory: /Users/kelvin.deng/Library/Caches/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [5]:
df = pd.read_csv("Resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [7]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad']]

# Create a Train Test Split

Use koi_disposition for the y values

In [41]:
target = df.koi_disposition

In [13]:
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state = 42)

In [14]:
X_train.head()

,koi_period,koi_impact,koi_duration,koi_depth,koi_prad
6122,6.768901,0.150,3.61600,123.1,1.24
6370,0.733726,0.291,2.30900,114.6,0.86
2879,7.652707,0.970,79.89690,641.1,3.21
107,7.953547,0.300,2.63120,875.4,2.25
29,4.959319,0.831,2.22739,9802.0,12.21


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [18]:
y.unique()

array(['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

In [19]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the Model

In [43]:
# Random Forest has overfitting issue, use cross validation instead
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
forest = RandomForestClassifier()
forest.fit(X_train_scaled, y_train)
scores = cross_val_score(forest, selected_features, target, cv=7)
scores


array([0.66266266, 0.6996997 , 0.74574575, 0.61561562, 0.7037037 ,
       0.75150301, 0.65831663])

In [46]:
print("Mean 7-Fold R Squared: {}".format(np.mean(scores)))

Mean 7-Fold R Squared: 0.6910352952437121


# Hyperparameter Tuning

Use GridSearchCV to tune the model's parameters

In [24]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {"n_neighbors" : [5,7,9,11,13,15,17,19,21]}
grid = GridSearchCV(model, param_grid, verbose=1)

In [26]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 7, 9, 11, 13, 15, 17, 19, 21]},
             verbose=1)

In [28]:
print(grid.best_params_)
print(grid.best_score_)

{'n_neighbors': 17}
0.6488695158602522


In [29]:
predictions = grid.predict(X_test_scaled)

In [30]:
print(predictions)

['CONFIRMED' 'CANDIDATE' 'FALSE POSITIVE' ... 'CONFIRMED' 'CANDIDATE'
 'FALSE POSITIVE']


# Save the Model

In [32]:
import joblib
filename = 'KNeighbors_planet.sav'
joblib.dump(model, 'KNeighbors_planet.sav')

['KNeighbors_planet.sav']